In [18]:
import json
with open("dataset.json", "r") as f:
    dataset_urls = json.load(f)
print(f"Loaded {len(dataset_urls)} links.")

Loaded 20113 links.


In [19]:

# Extract URLs and labels
urls = [item["url"] for item in dataset_urls]  # Add domain if needed
labels = [item["label"] for item in dataset_urls]

# Preprocess URLs (replace hyphens, slashes, etc.)
def preprocess(url):
    return url.lower().replace("-", " ").replace("/", " ").replace("_", " ")

processed_urls = [preprocess(url) for url in urls]
processed_urls[:5]

[' ',
 ' products nuon black slim fit mid rise jeans 300796748',
 ' products wes formals black relaxed fit trousers 300796731',
 ' products zuba off white silk blend embroidered cotton blend stole 300802883',
 ' products utsa black solid palazzos 300673565']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF vectorization
vectorizer = TfidfVectorizer(analyzer="char_wb", ngram_range=(3, 5))
X = vectorizer.fit_transform(processed_urls)


In [21]:
from sklearn.model_selection import train_test_split
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)


In [23]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
# XGBoost model
xgb = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
model = xgb.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb.predict(X_test)
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred))

/Users/reverie/Documents/web_crawler/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [08:51:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       866
           1       1.00      1.00      1.00      5168

    accuracy                           1.00      6034
   macro avg       1.00      1.00      1.00      6034
weighted avg       1.00      1.00      1.00      6034



In [25]:
import joblib
# Save the TF-IDF vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
# Save the trained model
joblib.dump(model, "model.pkl")


['model.pkl']

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
def preprocess_url(url: str) -> str:
    url = url.lower().replace("https://", "").replace("http://", "")
    url = url.replace(".", "/").replace("-", " ").replace("_", " ")
    return url

def predict_url_label(url, model: RandomForestClassifier, vectorizer: TfidfVectorizer) -> int:
    processed = preprocess_url(url)
    vectorized = vectorizer.transform([processed])  # note: list input
    prediction = model.predict(vectorized)[0]
    return prediction

for url in new_urls:
    response = predict_url_label(
        url=url,
        model=xgb,
        vectorizer=vectorizer,
    )
    if response == np.int64(0):
        print("Not a product page:", url)
